# Exploring `demand_gegis` outputs

### Jupyter notebook needs to be improved

The code below initialise the to be used packages. Further, `_sets_path_to_root` makes sure that we set the path to your pypsa-africa folder which we need for reading files. Please, also make sure you set up the `pypsa-africa` environment which is shared as `environment.yml` in the envs folder.

In [ ]:
import atlite
import cartopy.crs as ccrs
import xarray as xr
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer

import os
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")


plt.rcParams['figure.figsize'] = [7, 7]
%matplotlib inline

## Let's open the demand timeseries from GEGIS
Demand timeseries are calculated in GEGIS based on paramters such as GDP, population and temperature data. Since each of this parameters can be provided in high resolution, we can derive timeseries with GEGIS for arbitrary regions for multi-years.

GEGIS is currently written in Julia. A rewrite to Python can help improving GEGIS and including other factors such as humidity, building area, building archetype information. These information will help the AI to train more accurately regional differences for demand timeseries. Alternative approaches to GEGIS such as botton-up statistical ones can also be developed in parallel.


In [ ]:
demand_path = os.path.realpath("resources")+"/ssp2-2.6/2030/era5_2013/Afric.nc"

In [ ]:
demand_path = os.path.realpath("resources")+"/ssp2-2.6/2030/era5_2013/Africa.nc"
demand = xr.open_dataset(demand_path)

In [ ]:
demand

In [ ]:
demand["Electricity demand"].sel(region_code="MW", time=slice("2013-01-01", "2014-01-01")).plot()

In [ ]:
demand["Electricity demand"].sel(region_code="NG").plot()

# Updating demand of countries

In [ ]:
countries = ['MW', 'LS', 'GN', 'SZ', 'GQ', 'MG', 'MR', 'SL', 'CF', 'TD']
gdp = [12, 1.85, 15.6, 3.96, 10, 13.72, 7.7, 3.86, 2.3, 10] # Billion USD (2020)
base_country = ['NG']
base_gdp = 432

In [ ]:
ds = demand

In [ ]:
# Updates each missing load country to Nigeria based on GDP and Nigerias timeseries
for i in range(0, len(countries), 1):
    ds.loc[dict(region_code=countries[i])] = demand.loc[dict(region_code="NG")]*(gdp[i]/base_gdp)

In [ ]:
# Test if Malawi has now inputs
ds.loc[dict(region_code="MW")]["Electricity demand"]

## Export netcdf
uncomment the below function out to save Africa.nc (info: `.` in the path equals `os.getcwd()`)

In [ ]:
# ds.to_netcdf(path='./Africa.nc')

# TEST NEW FUNCTION
Combine all weather years and predictions to one .nc file.
Unsolved challenge, how to concat the .nc files
Probably best to wait for a GEGIS Python developments

## Reading the path of all .nc files

In [ ]:
path = os.path.realpath("resources/ssp2-2.6")
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))

#print all the file names
str_end = "Africa.nc"
filelist_filtered = [x for x in filelist if x.endswith(str_end)]
filelist_filtered

In [ ]:
# How to concat these nc files to one? Does it make sense